In [5]:
import numpy as np
import pandas as pd
import os
import env

## Acquire data from the cloud database.

In [11]:
def acquire_zillow():
    '''
    This function will retrieve zillow home data for 2017 properties. It will only get
    single family residential properties. the function will attempt to open the data from 
    a local csv file, if one is not found, it will download the data from the codeup
    database. An env file is needed in the local directory in order to run this file.
    '''
    if os.path.exists('zillow_2017.csv'):
        print('opening data from local file')
        df = pd.read_csv('zillow_2017.csv', index_col=0)
    else:
        # run sql query and write to csv
        print('local file not found')
        print('retrieving data from sql server')
        query = '''
SELECT *
FROM properties_2017
JOIN predictions_2017
	USING (parcelid)
LEFT JOIN airconditioningtype
	USING (airconditioningtypeid)
LEFT JOIN architecturalstyletype
	USING (architecturalstyletypeid)
LEFT JOIN buildingclasstype
	USING (buildingclasstypeid)
LEFT JOIN heatingorsystemtype
	USING (heatingorsystemtypeid)
LEFT JOIN propertylandusetype
	USING (propertylandusetypeid)
LEFT JOIN storytype
	USING (storytypeid)
LEFT JOIN typeconstructiontype
	USING (typeconstructiontypeid)
WHERE latitude IS NOT NULL
	AND longitude IS NOT NULL
    AND parcelid IN (
		SELECT parcelid FROM unique_properties)
;
        '''
        connection = env.get_db_url('zillow')
        df = pd.read_sql(query, connection)
        df.to_csv('zillow_2017.csv')
    
    # renaming column names to one's I like better
    df = df.rename(columns = {'bedroomcnt':'bedrooms', 
                              'bathroomcnt':'bathrooms', 
                              'calculatedfinishedsquarefeet':'area',
                              'garagecarcnt':'cars_garage',
                              'garagetotalsqft':'garage_sqft',
                              'lotsizesquarefeet':'lot_size',
                              'poolcnt':'pools',
                              'regionidcity':'region',
                              'yearbuilt':'year_built',
                              'taxvaluedollarcnt':'tax_value'
                              })
    
    df = df.drop(columns='id')
    
    return df

In [12]:
df = acquire_zillow()

local file not found
retrieving data from sql server


In [13]:
df 

,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,id,basementsqft,...,id,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,NaN,NaN,261.0,NaN,NaN,NaN,NaN,14297519,1727539,NaN,...,0,0.025595,2017-01-01,None,None,None,None,Single Family Residential,None,None
1,NaN,NaN,261.0,NaN,NaN,NaN,NaN,17052889,1387261,NaN,...,1,0.055619,2017-01-01,None,None,None,None,Single Family Residential,None,None
2,NaN,NaN,261.0,NaN,NaN,NaN,NaN,14186244,11677,NaN,...,2,0.005383,2017-01-01,None,None,None,None,Single Family Residential,None,None
3,NaN,NaN,261.0,2.0,NaN,NaN,NaN,12177905,2288172,NaN,...,3,-0.103410,2017-01-01,None,None,None,Central,Single Family Residential,None,None
4,NaN,NaN,266.0,2.0,NaN,NaN,1.0,10887214,1970746,NaN,...,4,0.006940,2017-01-01,Central,None,None,Central,Condominium,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77575,NaN,NaN,261.0,2.0,NaN,NaN,NaN,11000655,673515,NaN,...,77609,0.020615,2017-09-20,None,None,None,Central,Single Family Residential,None,None
77576,NaN,NaN,261.0,NaN,NaN,NaN,NaN,17239384,2968375,NaN,...,77610,0.013209,2017-09-21,None,None,None,None,Single Family Residential,None,None
77577,NaN,NaN,261.0,2.0,NaN,NaN,1.0,12773139,1843709,NaN,...,77611,0.037129,2017-09-21,Central,None,None,Central,Single Family Residential,None,None
77578,NaN,NaN,261.0,2.0,NaN,NaN,NaN,12826780,1187175,NaN,...,77612,0.007204,2017-09-25,None,None,None,Central,Single Family Residential,None,None


## Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [23]:
df = df.drop(columns='id')

In [26]:
def nulls_by_col(df):
    '''
    This function takes in a dataframe 
    and finds the number of missing values
    it returns a new dataframe with quantity and percent of missing values
    '''
    num_missing = df.isnull().sum()
    rows = df.shape[0]
    percent_missing = num_missing / rows * 100
    cols_missing = pd.DataFrame({'num_rows_missing': num_missing, 'percent_rows_missing': percent_missing})
    return cols_missing.sort_values(by='num_rows_missing', ascending=False)

def nulls_by_row(df):
    '''
    This function takes in a dataframe 
    and finds the number of missing values in a row
    it returns a new dataframe with quantity and percent of missing values
    '''
    num_missing = df.isnull().sum(axis=1)
    percent_miss = num_missing / df.shape[1] * 100
    rows_missing = pd.DataFrame({'num_cols_missing': num_missing, 'percent_cols_missing': percent_miss})
    rows_missing = df.merge(rows_missing,
                        left_index=True,
                        right_index=True)[['num_cols_missing', 'percent_cols_missing']]
    return rows_missing.sort_values(by='num_cols_missing', ascending=False)

def summarize(df):
    '''
    summarize will take in a single argument (a pandas dataframe) 
    and output to console various statistics on said dataframe, including:
    # .head()
    # .info()
    # .describe()
    # .value_counts()
    # observation of nulls in the dataframe
    '''
    print('SUMMARY REPORT')
    print('=====================================================\n\n')
    print('Dataframe head: ')
    print(df.head(3))
    print('=====================================================\n\n')
    print('Dataframe info: ')
    print(df.info())
    print('=====================================================\n\n')
    print('Dataframe Description: ')
    print(df.describe())
    num_cols = [col for col in df.columns if df[col].dtype != 'O']
    cat_cols = [col for col in df.columns if col not in num_cols]
    print('=====================================================')
    print('DataFrame value counts: ')
    for col in df.columns:
        if col in cat_cols:
            print(df[col].value_counts(), '\n')
        else:
            print(df[col].value_counts(bins=10, sort=False), '\n')
    print('=====================================================')
    print('nulls in dataframe by column: ')
    print(nulls_by_col(df))
    print('=====================================================')
    print('nulls in dataframe by row: ')
    print(nulls_by_row(df))
    print('=====================================================')

In [28]:
summarize(df)

SUMMARY REPORT


Dataframe head: 
   typeconstructiontypeid  storytypeid  propertylandusetypeid  \
0                     NaN          NaN                  261.0   
1                     NaN          NaN                  261.0   
2                     NaN          NaN                  261.0   

   heatingorsystemtypeid  buildingclasstypeid  architecturalstyletypeid  \
0                    NaN                  NaN                       NaN   
1                    NaN                  NaN                       NaN   
2                    NaN                  NaN                       NaN   

   airconditioningtypeid  parcelid  basementsqft  bathrooms  ...  \
0                    NaN  14297519           NaN        3.5  ...   
1                    NaN  17052889           NaN        1.0  ...   
2                    NaN  14186244           NaN        2.0  ...   

   censustractandblock  logerror  transactiondate  airconditioningdesc  \
0         6.059063e+13  0.025595       2017-01-01        

(7.819, 329.0]      1639
(329.0, 647.0]       607
(647.0, 965.0]        95
(965.0, 1283.0]       35
(1283.0, 1601.0]       9
(1601.0, 1919.0]       1
(1919.0, 2237.0]       2
(2237.0, 2555.0]       2
(2555.0, 2873.0]       1
(2873.0, 3191.0]       2
Name: yardbuildingsqft17, dtype: int64 

(11.143, 97.6]    24
(97.6, 183.2]     14
(183.2, 268.8]    10
(268.8, 354.4]     7
(354.4, 440.0]     4
(440.0, 525.6]     6
(525.6, 611.2]     2
(611.2, 696.8]     1
(696.8, 782.4]     1
(782.4, 868.0]     1
Name: yardbuildingsqft26, dtype: int64 

(1823.807, 1843.2]        1
(1843.2, 1862.4]          0
(1862.4, 1881.6]          3
(1881.6, 1900.8]         90
(1900.8, 1920.0]       1754
(1920.0, 1939.2]       6718
(1939.2, 1958.4]      18513
(1958.4, 1977.6]      20411
(1977.6, 1996.8]      19129
(1996.8, 2016.0]      10691
Name: year_built, dtype: int64 

(0.994, 1.5]    10397
(1.5, 2.0]       6764
(2.0, 2.5]          0
(2.5, 3.0]        437
(3.0, 3.5]          0
(3.5, 4.0]          0
(4.0, 4.5]   

### Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.
The output should look like the table below:

In [29]:
null_cols = nulls_by_col(df)

In [30]:
null_cols

,num_rows_missing,percent_rows_missing
buildingclassdesc,77565,99.980665
buildingclasstypeid,77565,99.980665
finishedsquarefeet13,77538,99.945862
storydesc,77530,99.935550
basementsqft,77530,99.935550
...,...,...
parcelid,0,0.000000
propertycountylandusecode,0,0.000000
bathrooms,0,0.000000
rawcensustractandblock,0,0.000000


### Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer.

In [31]:
df.propertylandusedesc.value_counts()

Single Family Residential                     52442
Condominium                                   19342
Duplex (2 Units, Any Combination)              2021
Planned Unit Development                       1947
Quadruplex (4 Units, Any Combination)           729
Triplex (3 Units, Any Combination)              539
Cluster Home                                    335
Mobile Home                                      74
Manufactured, Modular, Prefabricated Homes       59
Residential General                              42
Cooperative                                      29
Commercial/Office/Residential Mixed Used         15
Townhouse                                         6
Name: propertylandusedesc, dtype: int64

In [36]:
homes = ((df.propertylandusedesc =='Single Family Residential') |
          (df.propertylandusedesc == 'Mobile Home') |
         (df.propertylandusedesc =='Manufactured, Modular, Prefabricated Homes'))
df = df[homes]

In [38]:
df.propertylandusedesc.value_counts()

Single Family Residential                     52442
Mobile Home                                      74
Manufactured, Modular, Prefabricated Homes       59
Name: propertylandusedesc, dtype: int64

### Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

In [39]:
def handle_missing_values(df, prop_required_columns=0.5, prop_required_rows=0.75):
    '''
    This function takes in a dataframe, the percent of columns and rows
    that need to have values/non-nulls
    and returns the dataframe with the desired amount of nulls left.
    '''
    column_threshold = int(round(prop_required_columns * len(df.index), 0))
    df = df.dropna(axis=1, thresh=column_threshold)
    row_threshold = int(round(prop_required_rows * len(df.columns), 0))
    df = df.dropna(axis=0, thresh=row_threshold)
    return df

In [40]:
handle_missing_values(df, prop_required_columns=0.5, prop_required_rows=0.75)

,propertylandusetypeid,heatingorsystemtypeid,parcelid,bathrooms,bedrooms,buildingqualitytypeid,calculatedbathnbr,area,finishedsquarefeet12,fips,...,structuretaxvaluedollarcnt,tax_value,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,logerror,transactiondate,heatingorsystemdesc,propertylandusedesc
0,261.0,NaN,14297519,3.5,4.0,NaN,3.5,3100.0,3100.0,6059.0,...,485713.0,1023282.0,2016.0,537569.0,11013.72,6.059063e+13,0.025595,2017-01-01,None,Single Family Residential
1,261.0,NaN,17052889,1.0,2.0,NaN,1.0,1465.0,1465.0,6111.0,...,88000.0,464000.0,2016.0,376000.0,5672.48,6.111001e+13,0.055619,2017-01-01,None,Single Family Residential
2,261.0,NaN,14186244,2.0,3.0,NaN,2.0,1243.0,1243.0,6059.0,...,85289.0,564778.0,2016.0,479489.0,6488.30,6.059022e+13,0.005383,2017-01-01,None,Single Family Residential
3,261.0,2.0,12177905,3.0,4.0,8.0,3.0,2376.0,2376.0,6037.0,...,108918.0,145143.0,2016.0,36225.0,1777.51,6.037300e+13,-0.103410,2017-01-01,Central,Single Family Residential
6,261.0,2.0,12095076,3.0,4.0,9.0,3.0,2962.0,2962.0,6037.0,...,276684.0,773303.0,2016.0,496619.0,9516.26,6.037461e+13,-0.001011,2017-01-01,Central,Single Family Residential
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77575,261.0,2.0,11000655,2.0,2.0,6.0,2.0,1286.0,1286.0,6037.0,...,70917.0,354621.0,2016.0,283704.0,4478.43,6.037101e+13,0.020615,2017-09-20,Central,Single Family Residential
77576,261.0,NaN,17239384,2.0,4.0,NaN,2.0,1612.0,1612.0,6111.0,...,50683.0,67205.0,2016.0,16522.0,1107.48,6.111008e+13,0.013209,2017-09-21,None,Single Family Residential
77577,261.0,2.0,12773139,1.0,3.0,4.0,1.0,1032.0,1032.0,6037.0,...,32797.0,49546.0,2016.0,16749.0,876.43,6.037434e+13,0.037129,2017-09-21,Central,Single Family Residential
77578,261.0,2.0,12826780,2.0,3.0,6.0,2.0,1762.0,1762.0,6037.0,...,140000.0,522000.0,2016.0,382000.0,6317.15,6.037503e+13,0.007204,2017-09-25,Central,Single Family Residential
